In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
from sklearn.metrics import precision_score,accuracy_score,roc_auc_score
from sklearn.metrics import balanced_accuracy_score
from tqdm import tqdm
import pickle
def makefile(what,filename):
    with open(filename,"wb") as f3:
        pickle.dump(what,f3)

def readfile(filename):
    with open(filename,"rb") as f4:
        ans=pickle.load(f4)
    return ans
import random
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
#from imblearn.datasets import fetch_datasets
#fetch_dataset=fetch_datasets()
fetch_dataset=readfile("fetch_data.pkl")
data_name=[]
for word in fetch_dataset:
    print(word)
    data_name.append(word)

ecoli
optical_digits
satimage
pen_digits
abalone
sick_euthyroid
spectrometer
car_eval_34
isolet
us_crime
yeast_ml8
scene
libras_move
thyroid_sick
coil_2000
arrhythmia
solar_flare_m0
oil
car_eval_4
wine_quality
letter_img
yeast_me2
webpage
ozone_level
mammography
protein_homo
abalone_19


In [11]:
%%time
table_pos=[]
table_neg=[]
pos_models=[]
neg_models=[]
for e in range(1):
    for word in fetch_dataset:
        #print(word)
        X=fetch_dataset[word]["data"]
        Y=fetch_dataset[word]["target"]
        print(word,":",len(X[Y==1]),":",len(X[Y==-1]),":",len(X[0]))
        X=(X-X.min(axis=0))/(X.max(axis=0)-X.min(axis=0))
        X[pd.isnull(X)]=0
        random.seed(0)
        d=len(X[0])
        split=0.4
        random_value=1
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=split, random_state=e,stratify=Y)
        positive=X_train[y_train==1]
        negative=X_train[y_train==-1]
        print("positive")
        models=[]
        score_max=[]
        score_min=[]


        cov_list=["linear","poly","rbf","sigmoid"]
        for cov in cov_list:
            for n in range(10):
                OCSVM=OneClassSVM(kernel=cov,nu=(n*0.02+0.5))
                OCSVM.fit(positive)
                models.append(OCSVM)
                score=OCSVM.score_samples(positive)
                score_max.append(score.max())
                score_min.append(score.min())


        Ps=[1,1.5,2]
        for p in Ps:
            for n in range(10):
                
                LOF=LocalOutlierFactor(novelty=True,n_neighbors=n+1,p=p)
                LOF.fit(positive)
                models.append(LOF)
                score=LOF.score_samples(positive)
                score_max.append(score.max())
                score_min.append(score.min())

        for n in range(5):
            try:
                IF=IsolationForest(max_features=n+1)
                IF.fit(positive)
                models.append(IF)
                score=IF.score_samples(positive)
                score_max.append(score.max())
                score_min.append(score.min())
            except:
                break
                
        cov_list=["full","tied","diag","spherical"]
        for cov in cov_list:
            for n in range(10):
                GMM=GaussianMixture(n_components=n+1,covariance_type=cov)
                GMM.fit(positive)
                models.append(GMM)
                score=GMM.score_samples(positive)
                score_max.append(score.max())
                score_min.append(score.min())
        
    
        score_test=np.zeros(len(X_test))
        #score_test_k=np.zeros([6,len(X_test)])
        AUC_table=np.zeros(116)

        for n in range(115):
            score=models[n].score_samples(X_test)
            print(roc_auc_score(y_test,score))
            AUC_table[n]=roc_auc_score(y_test,score)
            if score_max[n]==score_min[n]:
                pass
            else:
                score_test+=(score-score_min[n])/(score_max[n]-score_min[n])
            
        print("ensemble:",roc_auc_score(y_test,score_test))
        AUC_table[115]=roc_auc_score(y_test,score_test)
        table_pos.append(AUC_table)
        pos_models.append(models)
        print("negative")
        models=[]
        score_max=[]
        score_min=[]
        cov_list=["linear","poly","rbf","sigmoid"]
        for cov in cov_list:
            for n in range(10):
                OCSVM=OneClassSVM(kernel=cov,nu=(n*0.02+0.5))
                OCSVM.fit(negative)
                models.append(OCSVM)
                score=OCSVM.score_samples(negative)
                score_max.append(score.max())
                score_min.append(score.min())

        Ps=[1,1.5,2]
        for p in Ps:
            for n in range(10):
                
                LOF=LocalOutlierFactor(novelty=True,n_neighbors=n+1,p=p)
                LOF.fit(negative)
                models.append(LOF)
                score=LOF.score_samples(negative)
                score_max.append(score.max())
                score_min.append(score.min())

        

        for n in range(5):
            try:
                IF=IsolationForest(max_features=n+1)
                IF.fit(negative)
                models.append(IF)
                score=IF.score_samples(negative)
                score_max.append(score.max())
                score_min.append(score.min())
            except:
                break
        
        cov_list=["full","tied","diag","spherical"]
        for cov in cov_list:
            for n in range(10):
                GMM=GaussianMixture(n_components=n+1,covariance_type=cov)
                GMM.fit(negative)
                models.append(GMM)
                score=GMM.score_samples(negative)
                score_max.append(score.max())
                score_min.append(score.min())
        
   
        score_test=np.zeros(len(X_test))
        #score_test_k=np.zeros([6,len(X_test)])
        AUC_table=np.zeros(116)
        for n in range(115):
            score=models[n].score_samples(X_test)
            print(roc_auc_score(y_test,-score))
            AUC_table[n]=roc_auc_score(y_test,-score)
            if score_max[n]==score_min[n]:
                pass
            else:
                score_test+=(score-score_min[n])/(score_max[n]-score_min[n])
            
        print("ensemble:",roc_auc_score(y_test,-score_test))
        AUC_table[115]=roc_auc_score(y_test,-score_test)
        table_neg.append(AUC_table)
        neg_models.append(models)
        #break

ecoli : 35 : 301 : 7
positive
0.935064935064935
0.935064935064935
0.935655253837072
0.935064935064935
0.935064935064935
0.935064935064935
0.935655253837072
0.935655253837072
0.935655253837072
0.935655253837072
0.9332939787485242
0.9332939787485242
0.9338842975206612
0.9344746162927982
0.9344746162927982
0.9344746162927982
0.9344746162927982
0.9344746162927982
0.9344746162927982
0.935064935064935
0.8913813459268004
0.8931523022432113
0.8925619834710743
0.8925619834710743
0.8931523022432113
0.8913813459268004
0.8913813459268004
0.8907910271546634
0.8902007083825265
0.8902007083825265
0.9403778040141677
0.9403778040141677
0.9409681227863046
0.9409681227863046
0.9409681227863046
0.9409681227863046
0.9409681227863046
0.9409681227863046
0.9409681227863046
0.9409681227863046
0.5832349468713105
0.8671782762691853
0.8813459268004723
0.8807556080283353
0.8842975206611571
0.8848878394332941
0.8890200708382527
0.8854781582054311
0.8872491145218419
0.885478158205431
0.5994687131050767
0.85950413223

<timed exec>:11: RuntimeWarning: invalid value encountered in true_divide


0.8600157413089298
0.859869000293482
0.8601380254884696
0.8602336294833828
0.8599601575909572
0.8595777416113053
0.8589463105751358
0.8593909803189171
0.8594732442215167
0.8592575793957826
0.8602113959961937
0.8608117001502984
0.8603092233398255
0.8602825431551987
0.8594888076625491
0.8590886048931459
0.8588306964417527
0.8592042190265291
0.8593465133445392
0.8592798128829718
0.973684444563023
0.9739779265939187
0.9740246169170157
0.9737600384194658
0.9739779265939187
0.9742847487171279
0.9742669619273766
0.9741535711427123
0.9740890940298641
0.9738334089271897
0.8587884528160934
0.8595732949138676
0.8600490915397133
0.8598645535960441
0.859915690616579
0.8592931529752852
0.8591553053547132
0.859226452513718
0.8594999244061435
0.8590841581957079
0.9118030913440587
0.9665986321958681
0.9782400860880623
0.9834827423672439
0.984525492916411
0.9836139199416594
0.9832826409825423
0.9825934028796812
0.9823354944282883
0.981984205330701
0.9298577946159388
0.9730085465524755
0.9844209955266223

<timed exec>:11: RuntimeWarning: invalid value encountered in true_divide


0.7174373293240465
0.7169847503912122
0.7190635451505016
0.7197309073056979
0.721894081188058
0.7237657635543555
0.7247246172256144
0.7261667331471878
0.7279156822435642
0.7285216777637997
0.7168850296094014
0.7155042803227885
0.7160872633549139
0.7166932588751496
0.7190328618330214
0.720827835905618
0.7224770642201835
0.7240342425823079
0.726174403976558
0.7267650578380535
0.7248780338130157
0.7262204289527784
0.7296876438280507
0.7314979595593876
0.7319965634684422
0.7322420300082844
0.7329630879690712
0.7333619710963148
0.7341674081801725
0.7343131539382037
0.7218557270412075
0.7219401061642785
0.7234359178914425
0.7250698045472678
0.7273557116995489
0.7282455279064773
0.7290049400141143
0.7304087017888373
0.7314596054125373
0.731873830198521
0.686589089012304
0.6654636249271271
0.7015165229664632
0.7245481881501027
0.710418520450431
0.6821208309042373
0.6693565708324384
0.6571791292074499
0.6572289895983554
0.6536543831119022
0.6787763492988862
0.6452970145132091
0.6872602865821853

<timed exec>:11: RuntimeWarning: invalid value encountered in true_divide


0.595906432748538
0.5953216374269007
0.5959064327485379
0.5947368421052632
0.5953216374269006
0.5970760233918129
0.5976608187134502
0.595906432748538
0.5970760233918129
0.5970760233918129
0.5929824561403509
0.5923976608187135
0.5929824561403508
0.5941520467836257
0.5941520467836257
0.5953216374269006
0.5964912280701755
0.5964912280701755
0.595906432748538
0.5959064327485379
0.5970760233918129
0.5900584795321637
0.5871345029239766
0.5894736842105264
0.5923976608187134
0.5941520467836257
0.5941520467836258
0.5929824561403509
0.5929824561403508
0.5912280701754387
0.5970760233918129
0.5964912280701755
0.5964912280701755
0.5953216374269006
0.595906432748538
0.5964912280701754
0.595906432748538
0.5947368421052632
0.5953216374269006
0.5935672514619883
0.741812865497076
0.6631578947368422
0.6877192982456142
0.6549707602339181
0.6485380116959064
0.6345029239766082
0.6087719298245614
0.616374269005848
0.6286549707602339
0.6707602339181287
0.6608187134502923
0.6763157894736842
0.6438596491228071


<timed exec>:11: RuntimeWarning: invalid value encountered in true_divide


0.5933147632311978
0.5894846796657381
0.5873955431754875
0.5868732590529248
0.586525069637883
0.5880919220055709
0.5891364902506964
0.5887883008356546
0.588091922005571
0.5872214484679665
0.586350974930362
0.5866991643454039
0.5868732590529248
0.5873955431754875
0.5858286908077994
0.5872214484679665
0.5854805013927576
0.5844359331476323
0.5846100278551531
0.5842618384401114
0.6504178272980501
0.6485027855153204
0.6479805013927576
0.6411908077994429
0.6340529247910863
0.629700557103064
0.6220403899721448
0.6166434540389972
0.612291086350975
0.6075905292479109
0.5760793871866295
0.5739902506963789
0.5736420612813371
0.5739902506963788
0.5717270194986072
0.5691155988857939
0.5692896935933147
0.570508356545961
0.5720752089136489
0.5724233983286908
0.7146587743732591
0.6774025069637885
0.6925487465181058
0.5630222841225626
0.5330779944289693
0.5200208913649025
0.50974930362117
0.5114902506963789
0.5074860724233984
0.48746518105849584
0.720316852367688
0.625
0.6259575208913648
0.557103064066

In [39]:
for value in np.round(np.array(table_pos).max(axis=1)*100,2):
    print(value)

94.1
99.02
93.1
99.88
84.19
87.03
97.49
99.72
97.53
77.33
62.53
62.75
97.09
78.85
66.75
74.18
65.09
79.86
99.91
56.97
99.88
88.14
85.02
83.58
89.59
90.29
77.87


In [40]:
for value in np.round(np.array(table_neg).max(axis=1)*100,2):
    print(value)

82.29
98.13
66.28
98.72
79.61
63.97
94.73
97.4
78.29
83.57
60.23
56.47
99.25
69.99
59.31
51.29
83.95
84.0
99.42
73.3
95.84
72.67
90.15
58.51
89.27
91.1
76.22


In [41]:
for value in np.round(np.array(table_neg).min(axis=1)*100,2):
    print(value)

7.73
19.26
17.41
55.73
26.1
37.59
53.11
34.16
18.58
57.66
46.57
42.28
48.81
35.08
38.65
32.92
48.8
53.39
11.58
50.97
41.69
28.57
37.88
33.14
13.25
19.44
34.71


In [42]:
print("positive_min")
for value in np.round(np.array(table_pos).min(axis=1)*100,2):
    print(value)
print("positive_avg")
for value in np.round(np.array(table_pos)[:,:115].mean(axis=1)*100,2):
    print(value)
print("positive_ALL")
for value in np.round(np.array(table_pos)[:,115]*100,2):
    print(value)

positive_min
58.32
78.67
58.36
32.36
32.6
66.74
18.66
68.97
70.32
50.23
41.21
54.49
50.75
58.36
47.38
52.05
38.21
45.42
47.19
40.01
69.89
53.11
50.28
65.99
41.01
59.97
53.58
positive_avg
87.88
93.66
79.61
88.38
67.52
75.41
69.28
96.06
88.94
70.91
51.06
59.12
78.77
71.45
56.31
59.29
54.64
61.67
96.22
48.64
89.3
76.57
70.18
75.8
67.95
80.27
67.38
positive_ALL
83.94
91.98
86.57
98.77
80.77
80.05
50.91
99.38
94.03
60.06
52.34
57.8
78.36
69.97
53.68
61.23
58.91
74.81
99.87
46.92
92.48
80.34
67.55
70.93
85.18
73.84
73.16


In [43]:

print("negative_avg")
for value in np.round(np.array(table_neg)[:,:115].mean(axis=1)*100,2):
    print(value)
print("negative_ALL")
for value in np.round(np.array(table_neg)[:,115]*100,2):
    print(value)

negative_avg
52.67
61.24
47.83
80.98
52.82
48.15
73.5
78.42
46.98
72.09
54.6
49.66
73.73
54.01
50.43
43.58
73.18
68.16
78.16
64.8
64.22
52.18
71.39
43.99
67.59
69.71
51.27
negative_ALL
41.5
90.43
58.85
94.49
74.03
61.49
58.32
97.2
76.15
63.38
49.51
54.89
77.61
66.42
52.65
47.78
78.23
68.44
98.88
71.24
88.16
52.82
83.0
48.22
56.2
80.3
43.94


In [46]:
print(np.round(np.array(table_neg)[:,:115].min(axis=1).mean()*100,2))
print(np.round(np.array(table_neg)[:,:115].mean(axis=1).mean()*100,2))
print(np.round(np.array(table_neg)[:,:115].max(axis=1).mean()*100,2))
print(np.round(np.array(table_neg)[:,115].mean()*100,2))

35.0
60.94
79.78
67.93


In [47]:
print(np.round(np.array(table_pos)[:,:115].min(axis=1).mean()*100,2))
print(np.round(np.array(table_pos)[:,:115].mean(axis=1).mean()*100,2))
print(np.round(np.array(table_pos)[:,115].max(axis=1).mean()*100,2))
print(np.round(np.array(table_pos)[:,115].mean()*100,2))

52.01
73.42
84.73
74.96


In [32]:
np.round(np.array(table_pos).min(axis=1)*100,2)

array([58.32, 78.67, 58.36, 32.36, 32.6 , 66.74, 18.66, 68.97, 70.32,
       50.23, 41.21, 54.49, 50.75, 58.36, 47.38, 52.05, 38.21, 45.42,
       47.19, 40.01, 69.89, 53.11, 50.28, 65.99, 41.01, 59.97, 53.58])

In [33]:
np.round(np.array(table_neg).min(axis=1)*100,2)

array([ 7.73, 19.26, 17.41, 55.73, 26.1 , 37.59, 53.11, 34.16, 18.58,
       57.66, 46.57, 42.28, 48.81, 35.08, 38.65, 32.92, 48.8 , 53.39,
       11.58, 50.97, 41.69, 28.57, 37.88, 33.14, 13.25, 19.44, 34.71])

In [36]:
np.round(np.array(table_pos)[:,115]*100,2)

array([83.94, 91.98, 86.57, 98.77, 80.77, 80.05, 50.91, 99.38, 94.03,
       60.06, 52.34, 57.8 , 78.36, 69.97, 53.68, 61.23, 58.91, 74.81,
       99.87, 46.92, 92.48, 80.34, 67.55, 70.93, 85.18, 73.84, 73.16])

In [37]:
np.round(np.array(table_neg)[:,115]*100,2)

array([41.5 , 90.43, 58.85, 94.49, 74.03, 61.49, 58.32, 97.2 , 76.15,
       63.38, 49.51, 54.89, 77.61, 66.42, 52.65, 47.78, 78.23, 68.44,
       98.88, 71.24, 88.16, 52.82, 83.  , 48.22, 56.2 , 80.3 , 43.94])

In [22]:
np.round(np.array(table_neg)*100,2).mean(axis=0)

array([46.89888889, 47.69962963, 47.12592593, 48.20296296, 47.62555556,
       46.06481481, 46.09925926, 47.90888889, 46.39333333, 47.14666667,
       49.84      , 49.79259259, 49.75148148, 49.77      , 49.77074074,
       49.74      , 49.73740741, 49.69962963, 49.67925926, 49.66222222,
       61.08259259, 61.12111111, 61.02814815, 61.04222222, 61.00222222,
       60.96703704, 60.83111111, 60.69296296, 60.68851852, 60.62259259,
       45.12037037, 46.05888889, 45.42      , 45.29851852, 45.03666667,
       45.02925926, 44.72888889, 44.64925926, 44.78592593, 44.84740741,
       62.71740741, 66.48      , 67.63111111, 67.77407407, 68.59074074,
       69.46703704, 70.25814815, 70.7562963 , 70.23851852, 69.51814815,
       62.63407407, 66.61740741, 66.70555556, 67.09037037, 68.38555556,
       69.31777778, 69.96259259, 70.53814815, 70.17481481, 69.38925926,
       61.86666667, 66.2237037 , 66.75185185, 67.03592593, 68.30037037,
       68.90444444, 69.12444444, 69.85666667, 69.69148148, 68.70

In [48]:
np.round(np.array(table_pos).mean(axis=0)*100,2)

array([[91.85, 91.2 , 91.44, 89.91, 89.61, 91.03],
       [86.38, 90.98, 92.5 , 91.26, 93.09, 92.61],
       [91.95, 92.99, 93.21, 92.72, 93.13, 93.08],
       [93.05, 97.65, 98.32, 98.85, 98.68, 98.32],
       [83.09, 84.04, 83.72, 84.27, 84.04, 84.03],
       [77.42, 82.38, 77.65, 78.88, 81.12, 80.56],
       [70.2 , 75.98, 79.12, 73.96, 78.29, 77.01],
       [70.46, 72.56, 71.16, 69.13, 70.76, 71.96],
       [89.  , 92.35, 89.86, 93.16, 91.81, 93.1 ],
       [71.02, 67.92, 69.69, 69.51, 65.13, 69.34],
       [56.17, 58.01, 52.14, 56.71, 56.47, 58.22],
       [55.15, 58.42, 57.16, 61.07, 57.64, 58.06],
       [83.81, 87.39, 86.49, 85.3 , 89.93, 87.99],
       [75.77, 74.51, 80.09, 79.46, 82.29, 79.21],
       [59.3 , 57.12, 57.01, 56.52, 56.9 , 57.77],
       [66.02, 68.6 , 72.34, 58.36, 61.23, 66.37],
       [51.78, 48.71, 46.32, 49.41, 52.68, 49.04],
       [58.48, 51.15, 50.89, 53.85, 54.49, 54.16],
       [52.31, 49.77, 48.18, 49.99, 48.35, 49.47],
       [43.46, 45.44, 45.28, 46